In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/argenisleon/Optimus/")

In [3]:
from optimus import Optimus

In [4]:
op= Optimus("dask-cudf", n_workers=4, threads_per_worker=2,
                                          processes=False, memory_limit="3G", comm=True, verbose= True)

/home/argenisleon/.conda/envs/rapids_blazing/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/argenisleon/.conda/envs/rapids_blazing/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


INFO:Just check that Spark and all necessary environments vars are present...
INFO:-----
INFO:Starting or setting Dask Client...
INFO:Config.ini not found


In [5]:
df = op.load.csv("data/crime.csv", sep=",", header=True, infer_schema='false', null_value="None", charset="latin1")

latin1


In [6]:
df = df.repartition(npartitions=1)

In [7]:
df = df.persist()

In [8]:
df.dtypes

INCIDENT_NUMBER         object
OFFENSE_CODE             int64
OFFENSE_CODE_GROUP      object
OFFENSE_DESCRIPTION     object
DISTRICT                object
REPORTING_AREA         float64
SHOOTING                object
OCCURRED_ON_DATE        object
YEAR                     int64
MONTH                    int64
DAY_OF_WEEK             object
HOUR                     int64
UCR_PART                object
STREET                  object
Lat                    float64
Long                   float64
Location                object
dtype: object

In [9]:
num_cols = ["OFFENSE_CODE","YEAR", "MONTH", "HOUR"]
# cc= ["INCIDENT_NUMBER","OFFENSE_CODE_GROUP","OFFENSE_DESCRIPTION","DISTRICT","REPORTING_AREA","REPORTING_AREA","SHOOTING","OCCURRED_ON_DATE","DAY_OF_WEEK","UCR_PART","STREET","Location"]


In [10]:
import numpy as np
bins = np.array([-30, 0, 3, 6, 9, 10, 19, 50])

### Using digitize

In [11]:
import dask
import cudf

# def histogram(ser, bins):
#     print(type(ser))
#     binned = ser.digitize(bins, right=False)
#     vc = binned.value_counts().sort_index()
#     return vc, cudf.Series(bins)

def hist_serie(serie, buckets):    
    def func(_serie):
        binned = _serie.digitize(bins, right=False)
        vc = binned.value_counts().sort_index()
        return vc, bins
            
    return dask.delayed(func)(serie)

delayed_tasks =[]
for num_col in num_cols:
    delayed_tasks.append(hist_serie(df[num_col],10))
%time dask.compute(*delayed_tasks)

CPU times: user 140 ms, sys: 36 ms, total: 176 ms
Wall time: 1.77 s


((8    319073
  dtype: int32, array([-30,   0,   3,   6,   9,  10,  19,  50])), (8    319073
  dtype: int32, array([-30,   0,   3,   6,   9,  10,  19,  50])), (2    45263
  3    74431
  4    99947
  5    26543
  6    72889
  dtype: int32, array([-30,   0,   3,   6,   9,  10,  19,  50])), (2     32068
  3     11308
  4     26919
  5     14740
  6    162967
  7     71071
  dtype: int32, array([-30,   0,   3,   6,   9,  10,  19,  50])))

### Using Cupy

In [17]:
import cupy as cp
import dask


def hist_serie(serie, buckets):    
    def func(_serie):
        arr = cp.fromDlpack(_serie.to_dlpack())
        return cp.histogram(arr, buckets)
            
    return dask.delayed(func)(serie)

In [18]:
delayed_tasks =[]
for num_col in num_cols:
    delayed_tasks.append(hist_serie(df[num_col],10))
%time print(dask.compute(*delayed_tasks))


((array([12600, 65983, 10820, 18794, 16752,  3015, 23563, 32552, 88665,
       46329]), array([ 111.,  483.,  855., 1227., 1599., 1971., 2343., 2715., 3087.,
       3459., 3831.])), (array([ 53388,      0,      0,  99114,      0,      0, 100886,      0,
            0,  65685]), array([2015. , 2015.3, 2015.6, 2015.9, 2016.2, 2016.5, 2016.8, 2017.1,
       2017.4, 2017.7, 2018. ])), (array([45263, 24146, 24086, 26199, 30568, 34556, 34823, 26543, 25737,
       47152]), array([ 1. ,  2.1,  3.2,  4.3,  5.4,  6.5,  7.6,  8.7,  9.8, 10.9, 12. ])), (array([32068,  7997,  8337, 36633, 32795, 35525, 53582, 41065, 33438,
       37633]), array([ 0. ,  2.3,  4.6,  6.9,  9.2, 11.5, 13.8, 16.1, 18.4, 20.7, 23. ])))
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 43 ms


### Using Optimus

In [19]:
%%time
df.cols.hist(num_cols)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 55.3 ms


{'OFFENSE_CODE': {'hist': [{'count': 169.0, 'lower': 111.0, 'upper': 297.0},
   {'count': 12431.0, 'lower': 297.0, 'upper': 483.0},
   {'count': 44188.0, 'lower': 483.0, 'upper': 669.0},
   {'count': 21795.0, 'lower': 669.0, 'upper': 855.0},
   {'count': 1548.0, 'lower': 855.0, 'upper': 1041.0},
   {'count': 9272.0, 'lower': 1041.0, 'upper': 1227.0},
   {'count': 16609.0, 'lower': 1227.0, 'upper': 1413.0},
   {'count': 2185.0, 'lower': 1413.0, 'upper': 1599.0},
   {'count': 216.0, 'lower': 1599.0, 'upper': 1785.0},
   {'count': 16536.0, 'lower': 1785.0, 'upper': 1971.0},
   {'count': 2759.0, 'lower': 1971.0, 'upper': 2157.0},
   {'count': 256.0, 'lower': 2157.0, 'upper': 2343.0},
   {'count': 2655.0, 'lower': 2343.0, 'upper': 2529.0},
   {'count': 20908.0, 'lower': 2529.0, 'upper': 2715.0},
   {'count': 2894.0, 'lower': 2715.0, 'upper': 2901.0},
   {'count': 29658.0, 'lower': 2901.0, 'upper': 3087.0},
   {'count': 63012.0, 'lower': 3087.0, 'upper': 3273.0},
   {'count': 25653.0, 'lower